In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, roc_auc_score, precision_recall_curve
)
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# Specify file path
# Use when in Google Colab
# file_path = "/content/drive/MyDrive/Datasets for Google Colab/student_success_enhanced.csv"

# Use when in local environment
file_path = "./student_success_enhanced.csv"

# Load dataset
df = pd.read_csv(file_path)

# Select features and target variable
df['Graduated'] = (df['Graduation_Rate'] >= 0.6).astype(int)

# Select features and target
features = [
    "GPA", "SAT_Score", "ACT_Score", "Family_Size", "Support_Center_Utilization",
    "Retention_Rate", "Study_Hours_Per_Week", "Work_Hours_Per_Week"
]
numerical_columns = ["GPA", "SAT_Score", "ACT_Score", "Family_Size", "Support_Center_Utilization",
                     "Retention_Rate", "Graduation_Age", "Study_Hours_Per_Week", "Student_Loan_Amount", "Distance_From_Home", "Work_Hours_Per_Week"]
nominal_columns = ["Marital_Status", "Life_Event", "Major"]
ordinal_columns = ["Income_Level", "Institution_Type", "Campus_Engagement", "First_Gen_Student", "Enrollment_Status"]
target = "Graduated"

Displaying the split between "Graduated" 1 and 0

In [6]:
df['Graduated'].value_counts()

Graduated
1    7159
0    2841
Name: count, dtype: int64

Display the dataset and information

In [7]:
# Show the first couple rows of the dataframe
df.head(13)

,Student_ID,GPA,SAT_Score,ACT_Score,Family_Size,Income_Level,Marital_Status,Support_Center_Utilization,Retention_Rate,Graduation_Rate,...,Graduation_Age,Major,Study_Hours_Per_Week,Student_Loan_Amount,Campus_Engagement,First_Gen_Student,Enrollment_Status,Distance_From_Home,Work_Hours_Per_Week,Graduated
0,1,2.73,1174,26,1,High,Married,0.23,0.72,0.62,...,22.3,STEM,13.3,30968.51,Low,False,Full-Time,42.0,16.9,1
1,2,2.61,1079,24,4,High,Married,0.15,0.68,0.63,...,23.0,STEM,25.1,18679.95,Low,True,Full-Time,5.0,4.4,1
2,3,2.81,1197,26,4,Low,Married,0.47,0.61,0.58,...,22.4,Education,15.1,39004.41,Low,False,Full-Time,62.9,9.5,0
3,4,3.35,1328,29,1,High,Divorced,0.00,0.90,0.90,...,22.5,STEM,13.1,15563.23,Low,False,Full-Time,93.1,4.2,1
4,5,3.02,1064,23,1,Middle,Single,0.22,0.61,0.66,...,22.9,Arts,21.8,6533.81,Low,False,Full-Time,63.1,15.5,1
5,6,2.43,1064,23,1,Middle,Divorced,0.26,0.54,0.58,...,22.5,STEM,13.0,28718.06,Medium,False,Full-Time,64.8,14.7,0
6,7,3.64,1336,30,4,Low,Single,0.28,0.71,0.65,...,22.0,Arts,17.1,21571.52,Low,False,Full-Time,65.2,8.7,1
7,8,2.80,1215,27,1,Low,Single,0.36,0.86,0.84,...,23.0,Health Sciences,15.3,8249.42,High,False,Full-Time,95.8,8.9,1
8,9,2.32,1029,23,1,Middle,Single,0.56,0.56,0.56,...,23.1,Arts,15.7,19196.86,Low,False,Part-Time,65.8,8.3,0
9,10,3.20,1181,26,1,Middle,Married,0.37,1.00,0.99,...,22.0,Business,3.1,16445.08,Medium,False,Full-Time,90.8,14.4,1


In [ ]:
# Display the unique values within a specific column
df['Life_Event'].unique()
# df['Income_Level'].value_counts()

In [ ]:
# Describe the different values of a specific column
df['Family_Size'].describe()

In [ ]:
# Replacing missing values in the Life_Event column with 'None'
df['Life_Event'] = df['Life_Event'].fillna('None')
# Display the unique values within a specific column
df['Life_Event'].unique()

In [9]:
# Prepare data
X = df.drop(columns=["Student_ID", "Graduation_Rate", "Graduated"])
y = df["Graduated"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(f"Number of training samples: {X_train.shape[0]}")
print(f"Number of testing samples: {X_test.shape[0]}")

Number of training samples: 6700
Number of testing samples: 3300


In [ ]:
# One-hot encode categorical (nominal) variables
X_train = pd.get_dummies(X_train, columns=nominal_columns, drop_first=True)

# Apply the same encoding to the test set
X_test = pd.get_dummies(X_test, columns=nominal_columns, drop_first=True)

# Handle categorical (ordinal) columns
le = LabelEncoder()
for col in ordinal_columns:
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

# Scale numerical features after encoding
scaler = StandardScaler()

# Fit and transform on the training data
X_train_scaled = scaler.fit_transform(X_train[numerical_columns])

# Transform the test data using the same scaler
X_test_scaled = scaler.transform(X_test[numerical_columns])

# Replace the scaled numerical columns back into the original dataframes
X_train[numerical_columns] = X_train_scaled
X_test[numerical_columns] = X_test_scaled

In [ ]:
print(X_train.shape)
print()
print(X_test.shape)

In [ ]:
X_train.head()

In [ ]:
print(X_train.head())

# LOGISTIC REGRESSION

In [ ]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train logistic regression model
model = LogisticRegression(max_iter=2000, solver='saga')
model.fit(X_train_scaled, y_train)

# Make predictions
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)
y_test_proba = model.predict_proba(X_test_scaled)[:, 1]

In [ ]:
# Evaluate model
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
conf_matrix = confusion_matrix(y_test, y_test_pred)
report = classification_report(y_test, y_test_pred)

# Print results
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", report)


In [ ]:
print(f"Predicted Class Distribution: {np.bincount(y_test_pred)}")

In [ ]:
# Visualization 1: Confusion Matrix Heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=["Not Graduated", "Graduated"], yticklabels=["Not Graduated", "Graduated"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# Visualization 2: ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_test_proba)
roc_auc = roc_auc_score(y_test, y_test_proba)

plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})", color='blue')
plt.plot([0, 1], [0, 1], 'k--', label="Random Guess")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.show()

# Visualization 3: Precision-Recall Curve (with zero_division parameter)
precision, recall, thresholds = precision_recall_curve(y_test, y_test_proba)

plt.figure(figsize=(6, 5))
plt.plot(recall, precision, label="Precision-Recall Curve", color='green')
plt.title("Precision-Recall Curve")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(loc="lower left")
plt.show()

# DECISION FOREST


In [ ]:
model = DecisionTreeClassifier(random_state=42,max_depth=4)
model.fit(X_train_scaled, y_train)

In [ ]:
from sklearn.tree import plot_tree

# Plot the trained decision tree
plt.figure(figsize=(40, 30))
plot_tree(model, filled=True, feature_names=X.columns, class_names=["Not Graduated", "Graduated"], rounded=True, fontsize=12)
plt.title("Decision Tree Visualization")
plt.show()

In [ ]:
# Make predictions
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)
y_test_proba = model.predict_proba(X_test_scaled)[:, 1]

# Evaluate model
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
conf_matrix = confusion_matrix(y_test, y_test_pred)
report = classification_report(y_test, y_test_pred)

print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", report)
print(f"Predicted Class Distribution: {np.bincount(y_test_pred)}")


In [ ]:
# Visualization 1: Confusion Matrix Heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=["Not Graduated", "Graduated"], yticklabels=["Not Graduated", "Graduated"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# Visualization 2: ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_test_proba)
roc_auc = roc_auc_score(y_test, y_test_proba)

plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})", color='blue')
plt.plot([0, 1], [0, 1], 'k--', label="Random Guess")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.show()

# Visualization 3: Precision-Recall Curve (with zero_division parameter)
precision, recall, thresholds = precision_recall_curve(y_test, y_test_proba)

plt.figure(figsize=(6, 5))
plt.plot(recall, precision, label="Precision-Recall Curve", color='green')
plt.title("Precision-Recall Curve")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(loc="lower left")
plt.show()

# RANDOM FOREST

In [ ]:
rf_model = RandomForestClassifier(random_state=42, n_estimators=100)
rf_model.fit(X_train_scaled, y_train)

# Predictions
y_pred = rf_model.predict(X_test_scaled)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
conf_matrix = confusion_matrix(y_test, y_pred)

# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=["Not Graduated", "Graduated"], yticklabels=["Not Graduated", "Graduated"])
plt.title("Confusion Matrix - Random Forest")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, roc_auc_score, precision_recall_curve
import plotly.graph_objects as go
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Load your dataset (assuming df is your DataFrame and you've already preprocessed it)
# Assuming your target is 'Graduation_Rate' (converted to 0 or 1 for classification)

'''
# Prepare features and target
X = df.drop(columns=['Student_ID', 'Graduation_Rate'])  # Dropping ID and target column
y = df['Graduation_Rate'].apply(lambda x: 1 if x >= 0.5 else 0)  # Binary target (0 or 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
'''

# Initialize the base model (Decision Tree as the weak learner for AdaBoost)
base_model = DecisionTreeClassifier(max_depth=1)

# Initialize AdaBoost with the base model
ada_boost = AdaBoostClassifier(estimator=base_model, n_estimators=50, random_state=42)

# Fit the AdaBoost model
ada_boost.fit(X_train, y_train)

# Predict on test data
y_pred = ada_boost.predict(X_test)
y_pred_proba = ada_boost.predict_proba(X_test)[:, 1]

# Model Evaluation

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{conf_matrix}")

# Classification Report
print(f"Classification Report:\n{classification_report(y_test, y_pred)}")

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Plot ROC curve using Plotly
fig_roc = go.Figure()

fig_roc.add_trace(go.Scatter(
    x=fpr,
    y=tpr,
    mode='lines',
    name=f'ROC Curve (AUC = {roc_auc:.2f})',
    line=dict(color='#FF6347', width=3)  # Tomato red color
))

fig_roc.add_trace(go.Scatter(
    x=[0, 1],
    y=[0, 1],
    mode='lines',
    name="Random Guess",
    line=dict(color='#D3D3D3', dash='dash', width=2)  # Light gray for random guess
))

fig_roc.update_layout(
    title="Receiver Operating Characteristic (ROC) Curve",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    showlegend=True,
    plot_bgcolor='rgba(0,0,0,0)',  # Transparent background
    paper_bgcolor='rgba(0,0,0,0)',  # Transparent background
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False)
)

fig_roc.show()

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)

# Plot Precision-Recall curve using Plotly
fig_prc = go.Figure()

fig_prc.add_trace(go.Scatter(
    x=recall,
    y=precision,
    mode='lines',
    name="Precision-Recall Curve",
    line=dict(color='#32CD32', width=3)  # Lime green color
))

fig_prc.update_layout(
    title="Precision-Recall Curve",
    xaxis_title="Recall",
    yaxis_title="Precision",
    showlegend=True,
    plot_bgcolor='rgba(0,0,0,0)',  # Transparent background
    paper_bgcolor='rgba(0,0,0,0)',  # Transparent background
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False)
)

fig_prc.show()

# Confusion Matrix Heatmap
fig_cm = ff.create_annotated_heatmap(
    z=conf_matrix,
    x=["Not Graduated", "Graduated"],
    y=["Not Graduated", "Graduated"],
    colorscale='RdBu',  # Red-Blue color scale for better contrast
    showscale=True
)

fig_cm.update_layout(
    title="Confusion Matrix",
    xaxis_title="Predicted",
    yaxis_title="Actual",
    xaxis=dict(tickmode='array', tickvals=[0, 1], ticktext=["Not Graduated", "Graduated"]),
    yaxis=dict(tickmode='array', tickvals=[0, 1], ticktext=["Not Graduated", "Graduated"]),
)

fig_cm.show()
